In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize

In [2]:
userLogInput = "../data/user_log_format1.csv"
userInfoInput = "../data/user_info_format1.csv"

In [3]:
userLog = pd.read_csv(userLogInput, low_memory=False)
userInfo = pd.read_csv(userInfoInput, low_memory=False)

In [4]:
userInfo.age_range.fillna(0,inplace=True)
userInfo.age_range.replace(8,7,inplace=True)

In [5]:
userInfo.gender.fillna(2, inplace=True)

In [6]:
userLog.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [7]:
userInfo.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [8]:
u_age = userInfo[['user_id','age_range']].copy()
u_age = u_age.drop_duplicates()
u_age.age_range = u_age.age_range.astype(int)


age_hot = label_binarize(np.array(u_age.age_range), classes=[0,1,2,3,4,5,6,7])

u_age['age_range_0'] = age_hot[:,0]
u_age['age_range_1'] = age_hot[:,1]
u_age['age_range_2'] = age_hot[:,2]
u_age['age_range_3'] = age_hot[:,3]
u_age['age_range_4'] = age_hot[:,4]
u_age['age_range_5'] = age_hot[:,5]
u_age['age_range_6'] = age_hot[:,6]
u_age['age_range_7'] = age_hot[:,7]
u_age.drop(columns = ['age_range'], inplace=True)

print(u_age.isnull().any())
print(u_age.shape)
u_age.head()


user_id        False
age_range_0    False
age_range_1    False
age_range_2    False
age_range_3    False
age_range_4    False
age_range_5    False
age_range_6    False
age_range_7    False
dtype: bool
(424170, 9)


,user_id,age_range_0,age_range_1,age_range_2,age_range_3,age_range_4,age_range_5,age_range_6,age_range_7
0,376517,0,0,0,0,0,0,1,0
1,234512,0,0,0,0,0,1,0,0
2,344532,0,0,0,0,0,1,0,0
3,186135,0,0,0,0,0,1,0,0
4,30230,0,0,0,0,0,1,0,0


In [9]:
u_gender = userInfo[['user_id','gender']].copy()
u_gender = u_gender.drop_duplicates(['user_id'])

gender_hot = label_binarize(np.array(u_gender.gender), classes=[0, 1, 2])
u_gender['gender_hot_0'] = gender_hot[:,0]
u_gender['gender_hot_1'] = gender_hot[:,1]
u_gender['gender_hot_2'] = gender_hot[:,2]
u_gender.drop(columns=['gender'],inplace=True)

print(u_gender.isnull().any())
print(u_gender.shape)
u_gender.head()

user_id         False
gender_hot_0    False
gender_hot_1    False
gender_hot_2    False
dtype: bool
(424170, 4)


,user_id,gender_hot_0,gender_hot_1,gender_hot_2
0,376517,0,1,0
1,234512,1,0,0
2,344532,1,0,0
3,186135,1,0,0
4,30230,1,0,0


In [10]:
u_actions = userLog[['user_id','action_type']].copy()
u_actions.action_type = 1
u_actions = u_actions.groupby(['user_id']).agg('sum').reset_index()
u_actions = u_actions.rename(index=str,columns={'action_type':'u_total_actions'})
print('total_actions',u_actions.shape)
u_actions.head()

total_actions (424170, 2)


,user_id,u_total_actions
0,1,33
1,2,63
2,3,68
3,4,50
4,5,173


In [11]:
u_actionsType = userLog[['user_id','action_type']].copy()
action_type_hot = label_binarize(np.array(u_actionsType.action_type), classes=[0, 1, 2, 3])

u_actionsType['u_action_type_hot_0'] = action_type_hot[:,0]
u_actionsType['u_action_type_hot_1'] = action_type_hot[:,1]
u_actionsType['u_action_type_hot_2'] = action_type_hot[:,2]
u_actionsType['u_action_type_hot_3'] = action_type_hot[:,3]

u_actionsType.drop(columns=['action_type'],inplace=True)

u_actionsType = u_actionsType.groupby(['user_id']).agg('sum').reset_index()
print(u_actionsType.shape)
u_actionsType.head()

(424170, 5)


,user_id,u_action_type_hot_0,u_action_type_hot_1,u_action_type_hot_2,u_action_type_hot_3
0,1,27,0,6,0
1,2,47,0,14,2
2,3,63,0,4,1
3,4,49,0,1,0
4,5,150,0,13,10


In [12]:
u_days = userLog[['user_id','time_stamp']].copy()
u_days.drop_duplicates(inplace=True)

u_days.time_stamp = 1
u_days = u_days.groupby(['user_id']).agg('sum').reset_index()
u_days = u_days.rename(index=str,columns={'time_stamp':'u_days'})
print('u_days',u_days.shape)
u_days.head()

u_days (424170, 2)


,user_id,u_days
0,1,5
1,2,9
2,3,13
3,4,10
4,5,30


In [13]:
u_items = userLog[['user_id','item_id']].copy()
u_items.drop_duplicates(inplace=True)

u_items.item_id = 1
u_items = u_items.groupby(['user_id']).agg('sum').reset_index()
u_items = u_items.rename(index=str,columns={'item_id':'u_items'})
print('u_items',u_items.shape)
u_items.head()

u_items (424170, 2)


,user_id,u_items
0,1,12
1,2,43
2,3,45
3,4,28
4,5,87


In [14]:
u_cats = userLog[['user_id','cat_id']].copy()
u_cats.drop_duplicates(inplace=True)

u_cats.cat_id = 1
u_cats = u_cats.groupby(['user_id']).agg('sum').reset_index()
u_cats = u_cats.rename(index=str,columns={'cat_id':'u_cats'})
print(u_cats.shape)
u_cats.head()

(424170, 2)


,user_id,u_cats
0,1,6
1,2,14
2,3,19
3,4,13
4,5,40


In [15]:
u12 = pd.merge(u_age, u_gender, on=['user_id'])

u123 = pd.merge(u12, u_actionsType, on=['user_id'])

u1234 = pd.merge(u123, u_actions, on=['user_id'])

u12345 = pd.merge(u1234, u_days, on=['user_id'])

u123456 = pd.merge(u12345, u_items, on=['user_id'])

u1234567 = pd.merge(u123456, u_cats, on=['user_id'])

userFeature = u1234567

print(userFeature.isnull().any())
print(userFeature.shape)
userFeature.head()

user_id                False
age_range_0            False
age_range_1            False
age_range_2            False
age_range_3            False
age_range_4            False
age_range_5            False
age_range_6            False
age_range_7            False
gender_hot_0           False
gender_hot_1           False
gender_hot_2           False
u_action_type_hot_0    False
u_action_type_hot_1    False
u_action_type_hot_2    False
u_action_type_hot_3    False
u_total_actions        False
u_days                 False
u_items                False
u_cats                 False
dtype: bool
(424170, 20)


,user_id,age_range_0,age_range_1,age_range_2,age_range_3,age_range_4,age_range_5,age_range_6,age_range_7,gender_hot_0,gender_hot_1,gender_hot_2,u_action_type_hot_0,u_action_type_hot_1,u_action_type_hot_2,u_action_type_hot_3,u_total_actions,u_days,u_items,u_cats
0,376517,0,0,0,0,0,0,1,0,0,1,0,73,0,5,1,79,10,39,17
1,234512,0,0,0,0,0,1,0,0,1,0,0,47,0,7,2,56,11,29,18
2,344532,0,0,0,0,0,1,0,0,1,0,0,64,0,8,19,91,14,54,25
3,186135,0,0,0,0,0,1,0,0,1,0,0,193,0,2,5,200,5,21,11
4,30230,0,0,0,0,0,1,0,0,1,0,0,29,0,3,0,32,13,24,13


In [16]:
s_actions = userLog[['seller_id','action_type']].copy()
s_actions.action_type = 1
s_actions = s_actions.groupby(['seller_id']).agg('sum').reset_index()
s_actions = s_actions.rename(index=str,columns={'action_type':'s_total_actions'})
print('total_actions',s_actions.shape)
s_actions.head()

total_actions (4995, 2)


,seller_id,s_total_actions
0,1,339140
1,2,2371
2,3,2645
3,4,3106
4,5,8192


In [17]:
s_actionsType = userLog[['seller_id','action_type']].copy()
action_type_hot = label_binarize(np.array(s_actionsType.action_type), classes=[0, 1, 2, 3])

s_actionsType['s_action_type_hot_0'] = action_type_hot[:,0]
s_actionsType['s_action_type_hot_1'] = action_type_hot[:,1]
s_actionsType['s_action_type_hot_2'] = action_type_hot[:,2]
s_actionsType['s_action_type_hot_3'] = action_type_hot[:,3]

s_actionsType.drop(columns=['action_type'],inplace=True)

s_actionsType = s_actionsType.groupby(['seller_id']).agg('sum').reset_index()
print(u_actionsType.shape)
s_actionsType.head()

(424170, 5)


,seller_id,s_action_type_hot_0,s_action_type_hot_1,s_action_type_hot_2,s_action_type_hot_3
0,1,308236,444,17705,12755
1,2,2030,8,189,144
2,3,2399,4,67,175
3,4,2646,2,294,164
4,5,7483,9,144,556


In [18]:
s_items = userLog[['seller_id','item_id']].copy()
s_items['items_of_the_seller'] = 1

s_items.drop_duplicates(['seller_id','item_id'],inplace=True)
s_items = s_items.groupby(['seller_id']).agg('sum').reset_index()
s_items.drop(columns=['item_id'],inplace=True)

print(s_items.shape)
s_items.head()

(4995, 2)


,seller_id,items_of_the_seller
0,1,2977
1,2,154
2,3,171
3,4,155
4,5,660


In [19]:
s_cats = userLog[['seller_id','cat_id']].copy()
s_cats['cats_of_the_seller'] = 1

s_cats.drop_duplicates(['seller_id','cat_id'],inplace=True)
s_cats = s_cats.groupby(['seller_id']).agg('sum').reset_index()
s_cats.drop(columns=['cat_id'],inplace=True)

print(s_cats.shape)
s_cats.head()

(4995, 2)


,seller_id,cats_of_the_seller
0,1,44
1,2,10
2,3,4
3,4,7
4,5,23


In [20]:
s12 = pd.merge(s_actions, s_actionsType, on=['seller_id'])

s123 = pd.merge(s12, s_items, on=['seller_id'])

s1234 = pd.merge(s123, s_cats, on=['seller_id'])



In [21]:
sellerFeature = s1234
print(sellerFeature.isnull().any())
print(sellerFeature.shape)
sellerFeature.head()

seller_id              False
s_total_actions        False
s_action_type_hot_0    False
s_action_type_hot_1    False
s_action_type_hot_2    False
s_action_type_hot_3    False
items_of_the_seller    False
cats_of_the_seller     False
dtype: bool
(4995, 8)


,seller_id,s_total_actions,s_action_type_hot_0,s_action_type_hot_1,s_action_type_hot_2,s_action_type_hot_3,items_of_the_seller,cats_of_the_seller
0,1,339140,308236,444,17705,12755,2977,44
1,2,2371,2030,8,189,144,154,10
2,3,2645,2399,4,67,175,171,4
3,4,3106,2646,2,294,164,155,7
4,5,8192,7483,9,144,556,660,23


In [22]:
us_actions = userLog[['user_id','seller_id','action_type']].copy()
us_actions.action_type = 1
us_actions = us_actions.groupby(['user_id','seller_id']).agg('sum').reset_index()
us_actions = us_actions.rename(index=str,columns={'action_type':'us_actions'})
print(us_actions.shape)
us_actions.head()

(14058666, 3)


,user_id,seller_id,us_actions
0,1,471,1
1,1,739,1
2,1,925,4
3,1,1019,14
4,1,1156,1


In [23]:
us_actions_type = userLog[['user_id','seller_id','action_type']].copy()

action_type_hot = label_binarize(np.array(us_actions_type.action_type), classes=[0, 1, 2, 3])

us_actions_type['us_action_type_hot_0'] = action_type_hot[:,0]
us_actions_type['us_action_type_hot_1'] = action_type_hot[:,1]
us_actions_type['us_action_type_hot_2'] = action_type_hot[:,2]
us_actions_type['us_action_type_hot_3'] = action_type_hot[:,3]

us_actions_type.drop(columns=['action_type'],inplace=True)


us_actions_type = us_actions_type.groupby(['user_id','seller_id']).agg('sum').reset_index()


print(us_actions_type.shape)
us_actions_type.head()

(14058666, 6)


,user_id,seller_id,us_action_type_hot_0,us_action_type_hot_1,us_action_type_hot_2,us_action_type_hot_3
0,1,471,1,0,0,0
1,1,739,1,0,0,0
2,1,925,3,0,1,0
3,1,1019,10,0,4,0
4,1,1156,1,0,0,0


In [24]:
us_days = userLog[['user_id','seller_id','time_stamp']].copy()
us_days.drop_duplicates(inplace=True)

us_days.time_stamp = 1
us_days = us_days.groupby(['user_id','seller_id']).agg('sum').reset_index()
us_days = us_days.rename(index=str,columns={'time_stamp':'us_days'})

print(us_days.shape)
us_days.head()

(14058666, 3)


,user_id,seller_id,us_days
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


In [25]:
us_items = userLog[['user_id','seller_id','item_id']].copy()
us_items.drop_duplicates(inplace=True)

us_items.item_id = 1
us_items = us_items.groupby(['user_id','seller_id']).agg('sum').reset_index()
us_items = us_items.rename(index=str,columns={'item_id':'us_items'})

print(us_items.shape)
us_items.head()

(14058666, 3)


,user_id,seller_id,us_items
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


In [26]:
us_cats = userLog[['user_id','seller_id','cat_id']].copy()
us_cats.drop_duplicates(inplace = True)

us_cats.cat_id = 1
us_cats = us_cats.groupby(['user_id','seller_id']).agg('sum').reset_index()
us_cats = us_cats.rename(index=str,columns={'cat_id':'us_cats'})

print(us_cats.shape)
us_cats.head()

(14058666, 3)


,user_id,seller_id,us_cats
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


In [27]:
us1 = pd.merge(us_actions, us_actions_type, on=['user_id', 'seller_id'], how='left')

us12 = pd.merge(us1, us_items, on=['user_id','seller_id'], how='left')

us123 = pd.merge(us12, us_days, on = ['user_id', 'seller_id'], how='left')

us1234 = pd.merge(us123, us_cats, on = ['user_id', 'seller_id'], how='left')




In [28]:
usFeature = us1234


In [29]:
print(usFeature.isnull().any())
print(usFeature.shape)
usFeature.head()

user_id                 False
seller_id               False
us_actions              False
us_action_type_hot_0    False
us_action_type_hot_1    False
us_action_type_hot_2    False
us_action_type_hot_3    False
us_items                False
us_days                 False
us_cats                 False
dtype: bool
(14058666, 10)


,user_id,seller_id,us_actions,us_action_type_hot_0,us_action_type_hot_1,us_action_type_hot_2,us_action_type_hot_3,us_items,us_days,us_cats
0,1,471,1,1,0,0,0,1,1,1
1,1,739,1,1,0,0,0,1,1,1
2,1,925,4,3,0,1,0,1,1,1
3,1,1019,14,10,0,4,0,1,1,1
4,1,1156,1,1,0,0,0,1,1,1


In [30]:
userFeature.to_csv("../data/userFeatures.csv")


In [31]:
sellerFeature.to_csv("../data/sellerFeatures.csv")


In [32]:
usFeature.to_csv("../data/usFeatures.csv")

In [26]:
userFeature=pd.read_csv("../data/userFeatures.csv")
sellerFeature=pd.read_csv("../data/sellerFeatures.csv")
usFeature=pd.read_csv("../data/usFeatures.csv")

In [27]:
allTrainData = usFeature[['user_id','seller_id']].copy()
allTrainData.drop_duplicates(inplace=True)
print(allTrainData.shape)

allTrainData = pd.merge(allTrainData, userFeature, on=['user_id'])
print(allTrainData.shape)


allTrainData = pd.merge(allTrainData, sellerFeature, on=['seller_id'])
print(allTrainData.shape)


allTrainData = pd.merge(allTrainData, usFeature, on=['user_id','seller_id'])
print(allTrainData.shape)


(14058666, 2)
(14058666, 22)
(14058666, 30)
(14058666, 39)


In [28]:
from sklearn import preprocessing

In [29]:
allTrainData = allTrainData.sort_values(by=['user_id','seller_id']).reset_index()

In [30]:
allTrainData.head()

,index,user_id,seller_id,Unnamed: 0_x,age_range_0,age_range_1,age_range_2,age_range_3,age_range_4,age_range_5,...,cats_of_the_seller,Unnamed: 0,us_actions,us_action_type_hot_0,us_action_type_hot_1,us_action_type_hot_2,us_action_type_hot_3,us_items,us_days,us_cats
0,0,1,471,415932,0,0,0,1,0,0,...,20,0,1,1,0,0,0,1,1,1
1,7128,1,739,415932,0,0,0,1,0,0,...,17,1,1,1,0,0,0,1,1,1
2,10301,1,925,415932,0,0,0,1,0,0,...,14,2,4,3,0,1,0,1,1,1
3,15209,1,1019,415932,0,0,0,1,0,0,...,8,3,14,10,0,4,0,1,1,1
4,18148,1,1156,415932,0,0,0,1,0,0,...,8,4,1,1,0,0,0,1,1,1


In [31]:
allTrainData.isnull().any()

index                   False
user_id                 False
seller_id               False
Unnamed: 0_x            False
age_range_0             False
age_range_1             False
age_range_2             False
age_range_3             False
age_range_4             False
age_range_5             False
age_range_6             False
age_range_7             False
gender_hot_0            False
gender_hot_1            False
gender_hot_2            False
u_action_type_hot_0     False
u_action_type_hot_1     False
u_action_type_hot_2     False
u_action_type_hot_3     False
u_total_actions         False
u_days                  False
u_items                 False
u_cats                  False
Unnamed: 0_y            False
s_total_actions         False
s_action_type_hot_0     False
s_action_type_hot_1     False
s_action_type_hot_2     False
s_action_type_hot_3     False
items_of_the_seller     False
cats_of_the_seller      False
Unnamed: 0              False
us_actions              False
us_action_

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score


In [33]:
trainPath = "../data/train_format1.csv"
testPath = "../data/test_format1.csv"

In [34]:
trainData = pd.read_csv(trainPath, low_memory=False)
testData = pd.read_csv(testPath, low_memory=False)

In [35]:
trainData.shape

(260864, 3)

In [36]:
trainData.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [37]:
testData.shape

(261477, 3)

In [38]:
testData.head()

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN


In [39]:
trainData = trainData.rename(index=str,columns={'merchant_id':'seller_id'})
testData = testData.rename(index=str,columns={'merchant_id':'seller_id'})

In [40]:
dataset = allTrainData.iloc[:,1:]

In [41]:
t_dataset = pd.merge(dataset,trainData,on=['user_id','seller_id'],how='right')
t_dataset.drop(columns=['user_id','seller_id'],inplace=True)
print('t_dataset',t_dataset.shape)
t_dataset.isnull().any()

t_dataset (260864, 38)


Unnamed: 0_x            False
age_range_0             False
age_range_1             False
age_range_2             False
age_range_3             False
age_range_4             False
age_range_5             False
age_range_6             False
age_range_7             False
gender_hot_0            False
gender_hot_1            False
gender_hot_2            False
u_action_type_hot_0     False
u_action_type_hot_1     False
u_action_type_hot_2     False
u_action_type_hot_3     False
u_total_actions         False
u_days                  False
u_items                 False
u_cats                  False
Unnamed: 0_y            False
s_total_actions         False
s_action_type_hot_0     False
s_action_type_hot_1     False
s_action_type_hot_2     False
s_action_type_hot_3     False
items_of_the_seller     False
cats_of_the_seller      False
Unnamed: 0              False
us_actions              False
us_action_type_hot_0    False
us_action_type_hot_1    False
us_action_type_hot_2    False
us_action_

In [42]:
c_dataset = pd.merge(dataset,testData,on=['user_id','seller_id'],how='right')

c_dataset.drop(columns=['user_id','seller_id'],inplace=True)
X_challenge = c_dataset.iloc[:,0:-1]

X_challenge.count()

Unnamed: 0_x            261477
age_range_0             261477
age_range_1             261477
age_range_2             261477
age_range_3             261477
age_range_4             261477
age_range_5             261477
age_range_6             261477
age_range_7             261477
gender_hot_0            261477
gender_hot_1            261477
gender_hot_2            261477
u_action_type_hot_0     261477
u_action_type_hot_1     261477
u_action_type_hot_2     261477
u_action_type_hot_3     261477
u_total_actions         261477
u_days                  261477
u_items                 261477
u_cats                  261477
Unnamed: 0_y            261477
s_total_actions         261477
s_action_type_hot_0     261477
s_action_type_hot_1     261477
s_action_type_hot_2     261477
s_action_type_hot_3     261477
items_of_the_seller     261477
cats_of_the_seller      261477
Unnamed: 0              261477
us_actions              261477
us_action_type_hot_0    261477
us_action_type_hot_1    261477
us_actio

In [48]:
X_challenge=X_challenge.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])

In [49]:
X_challenge.shape

(261477, 34)

In [50]:
X_challenge.head()


,age_range_0,age_range_1,age_range_2,age_range_3,age_range_4,age_range_5,age_range_6,age_range_7,gender_hot_0,gender_hot_1,...,items_of_the_seller,cats_of_the_seller,us_actions,us_action_type_hot_0,us_action_type_hot_1,us_action_type_hot_2,us_action_type_hot_3,us_items,us_days,us_cats
0,-0.248537,-0.529061,-0.007574,-0.369152,1.667641,-0.494422,-0.329540,-0.30069,-0.14114,0.69522,...,244,8,3,0,0,3,0,2,1,1
1,-0.248537,-0.529061,-0.007574,-0.369152,1.667641,-0.494422,-0.329540,-0.30069,-0.14114,0.69522,...,150,26,2,0,0,2,0,1,1,1
2,-0.327672,-0.529061,-0.007574,-0.369152,1.667641,-0.494422,-0.329540,-0.30069,-0.14114,0.69522,...,35,6,6,4,0,1,1,2,2,2
3,1.420611,-0.529061,-0.007574,-0.369152,-0.599649,-0.494422,3.034528,-0.30069,-0.14114,0.69522,...,99,9,11,8,0,3,0,3,1,1
4,-1.572408,-0.529061,-0.007574,-0.369152,-0.599649,2.022564,-0.329540,-0.30069,-0.14114,0.69522,...,59,13,11,9,0,2,0,4,3,3


In [55]:
X_challenge_scaled = preprocessing.scale(X_challenge)
X_challenge.age_range_0 = X_challenge_scaled[:,0]       
X_challenge.age_range_1 = X_challenge_scaled[:,1]
X_challenge.age_range_2 = X_challenge_scaled[:,2]
X_challenge.age_range_3 = X_challenge_scaled[:,3]
X_challenge.age_range_4 = X_challenge_scaled[:,4]
X_challenge.age_range_5 = X_challenge_scaled[:,5]
X_challenge.age_range_6 = X_challenge_scaled[:,6]
X_challenge.age_range_7 = X_challenge_scaled[:,7]
X_challenge.gender_hot_0= X_challenge_scaled[:,8]
X_challenge.gender_hot_1= X_challenge_scaled[:,9]
X_challenge.gender_hot_2= X_challenge_scaled[:,10]
X_challenge.u_action_type_hot_0= X_challenge_scaled[:,11]
X_challenge.u_action_type_hot_1= X_challenge_scaled[:,12]
X_challenge.u_action_type_hot_2= X_challenge_scaled[:,13]
X_challenge.u_action_type_hot_3= X_challenge_scaled[:,14]
X_challenge.u_total_actions= X_challenge_scaled[:,15]
X_challenge.u_days  = X_challenge_scaled[:,16]
X_challenge.u_items = X_challenge_scaled[:,17]
X_challenge.u_cats  = X_challenge_scaled[:,18]
X_challenge.s_total_actions= X_challenge_scaled[:,19]
X_challenge.s_action_type_hot_0= X_challenge_scaled[:,20]
X_challenge.s_action_type_hot_1= X_challenge_scaled[:,21]
X_challenge.s_action_type_hot_2= X_challenge_scaled[:,22]
X_challenge.s_action_type_hot_3 = X_challenge_scaled[:,23]
X_challenge.items_of_the_seller = X_challenge_scaled[:,24]
X_challenge.cats_of_the_seller  = X_challenge_scaled[:,25]
X_challenge.us_actions          = X_challenge_scaled[:,26]
X_challenge.us_action_type_hot_0= X_challenge_scaled[:,27]
X_challenge.us_action_type_hot_1= X_challenge_scaled[:,28]
X_challenge.us_action_type_hot_2= X_challenge_scaled[:,29]
X_challenge.us_action_type_hot_3= X_challenge_scaled[:,30]
X_challenge.us_items            = X_challenge_scaled[:,31]
X_challenge.us_days             = X_challenge_scaled[:,32]
X_challenge.us_cats             = X_challenge_scaled[:,33]

In [56]:
X_challenge_scaled.shape

(261477, 34)

In [57]:
X_challenge_scaled

array([[-0.24853684, -0.52906089, -0.00757428, ..., -0.30007207,
        -0.52252884, -0.47112563],
       [-0.24853684, -0.52906089, -0.00757428, ..., -0.42498323,
        -0.52252884, -0.47112563],
       [-0.32767194, -0.52906089, -0.00757428, ..., -0.30007207,
         0.06365157,  0.06183498],
       ...,
       [-1.00259264, -0.52906089, -0.00757428, ..., -0.17516091,
         0.64983197,  0.06183498],
       [ 0.11684395, -0.52906089, -0.00757428, ..., -0.30007207,
        -0.52252884, -0.47112563],
       [-0.68384111,  1.89014159, -0.00757428, ..., -0.05024975,
        -0.52252884, -0.47112563]])

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from sklearn.datasets import make_multilabel_classification

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [59]:
X = t_dataset.iloc[:,0:-1]
Y = t_dataset.iloc[:,-1]

In [66]:
X=X.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])

In [67]:
X_scaled = preprocessing.scale(X)
X.age_range_0 = X_scaled[:,0]       
X.age_range_1 = X_scaled[:,1]
X.age_range_2 = X_scaled[:,2]
X.age_range_3 = X_scaled[:,3]
X.age_range_4 = X_scaled[:,4]
X.age_range_5 = X_scaled[:,5]
X.age_range_6 = X_scaled[:,6]
X.age_range_7 = X_scaled[:,7]
X.gender_hot_0= X_scaled[:,8]
X.gender_hot_1= X_scaled[:,9]
X.gender_hot_2= X_scaled[:,10]
X.u_action_type_hot_0= X_scaled[:,11]
X.u_action_type_hot_1= X_scaled[:,12]
X.u_action_type_hot_2= X_scaled[:,13]
X.u_action_type_hot_3= X_scaled[:,14]
X.u_total_actions= X_scaled[:,15]
X.u_days  = X_scaled[:,16]
X.u_items = X_scaled[:,17]
X.u_cats  = X_scaled[:,18]
X.s_total_actions= X_scaled[:,19]
X.s_action_type_hot_0= X_scaled[:,20]
X.s_action_type_hot_1= X_scaled[:,21]
X.s_action_type_hot_2= X_scaled[:,22]
X.s_action_type_hot_3 = X_scaled[:,23]
X.items_of_the_seller = X_scaled[:,24]
X.cats_of_the_seller  = X_scaled[:,25]
X.us_actions          = X_scaled[:,26]
X.us_action_type_hot_0= X_scaled[:,27]
X.us_action_type_hot_1= X_scaled[:,28]
X.us_action_type_hot_2= X_scaled[:,29]
X.us_action_type_hot_3= X_scaled[:,30]
X.us_items            = X_scaled[:,31]
X.us_days             = X_scaled[:,32]
X.us_cats             = X_scaled[:,33]

In [68]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=100)

In [69]:
X_test.head()

,age_range_0,age_range_1,age_range_2,age_range_3,age_range_4,age_range_5,age_range_6,age_range_7,gender_hot_0,gender_hot_1,...,items_of_the_seller,cats_of_the_seller,us_actions,us_action_type_hot_0,us_action_type_hot_1,us_action_type_hot_2,us_action_type_hot_3,us_items,us_days,us_cats
94210,-1.510702,-0.529138,-0.00706,-0.367411,-0.601872,-0.494376,3.030320,-0.301226,-0.137494,-1.445329,...,-0.479875,-0.387937,-0.560648,-0.622120,-0.484127,-0.394925,-0.371836,-0.112848,-0.394717,-0.271128
32665,-1.695640,-0.529138,-0.00706,-0.367411,-0.601872,-0.494376,-0.329998,3.319763,-0.137494,-1.445329,...,-0.601435,-0.388475,-0.544799,-0.666894,-1.294658,-0.243545,-0.266754,-0.112848,0.769007,-0.271128
185331,-1.244393,-0.529138,-0.00706,-0.367411,-0.601872,2.022750,-0.329998,-0.301226,-0.137494,0.691884,...,-0.045646,-0.289199,-0.415368,-0.711669,0.729092,-0.243545,-0.214212,-0.112848,-0.394717,-0.271128
72217,-0.999916,-0.529138,-0.00706,-0.367411,-0.601872,-0.494376,3.030320,-0.301226,-0.137494,-1.445329,...,-0.333051,-0.322964,-0.381029,-0.666894,-0.774707,-0.243545,-0.319295,9.392461,-0.394717,-0.271128
6235,-1.664319,-0.529138,-0.00706,-0.367411,-0.601872,-0.494376,-0.329998,3.319763,-0.137494,-1.445329,...,-0.541695,-0.352423,-0.420650,0.676349,-1.645445,-0.193085,-0.214212,-0.112848,0.769007,-0.271128


In [70]:
cls = RandomForestClassifier()

In [71]:
cls.fit(X_train, Y_train)
print(roc_auc_score(Y_test,cls.predict_proba(X_test)[:,1]))
cls.predict_proba(X_test)

0.5707141832097584


array([[1. , 0. ],
       [0.9, 0.1],
       [0.5, 0.5],
       ...,
       [1. , 0. ],
       [0.8, 0.2],
       [1. , 0. ]])

In [72]:
pred = cls.predict_proba(X_challenge)

In [73]:
pred

array([[0.8, 0.2],
       [0.8, 0.2],
       [0.7, 0.3],
       ...,
       [0.8, 0.2],
       [0.8, 0.2],
       [0.8, 0.2]])

In [74]:
output = pd.read_csv("../data/test_format1.csv")
output.head()

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN


In [75]:
output.prob = pred[:,1]

In [76]:
output.head()

,user_id,merchant_id,prob
0,163968,4605,0.2
1,360576,1581,0.2
2,98688,1964,0.3
3,98688,3645,0.2
4,295296,3361,0.2


In [77]:
output.to_csv("../data/RFCoutput0.csv",index=False)

In [111]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
print(knn.predict_proba(X_test))

print(roc_auc_score(Y_test,knn.predict_proba(X_test)[:,1]))



In [ ]:
knn_result = knn.predict_proba(X_challenge)


In [114]:
output = pd.read_csv("../data/test_format1.csv")
output.head()
output.prob = knn_result[:,1]
output.to_csv("../data/knn_result.csv",index=False)

In [159]:

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [160]:
xgb = XGBClassifier()
xgb.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [161]:
print(roc_auc_score(Y_test,xgb.predict_proba(X_test)[:,1]))
xgb.predict_proba(X_test)


0.6464393275298654


array([[0.96314   , 0.03686002],
       [0.9306473 , 0.06935268],
       [0.9213972 , 0.0786028 ],
       ...,
       [0.9530295 , 0.04697051],
       [0.95378417, 0.04621582],
       [0.9611498 , 0.03885021]], dtype=float32)

In [162]:
xgb_pred = xgb.predict_proba(X_challenge)

In [163]:
output = pd.read_csv("../data/test_format1.csv")
output.prob = xgb_pred[:,1]
output.to_csv("../data/XGBresult.csv",index=False)

In [120]:
gbm = GradientBoostingClassifier(max_features='sqrt',
                                  learning_rate=0.1,
                                  min_samples_leaf=20,
                                  subsample=0.8,
                                  random_state=10,
                                  n_estimators=100)
gbm.fit(X_train,Y_train)
y_predprob = gbm.predict_proba(X_test)
print(roc_auc_score(Y_test,y_predprob[:,1]))

0.6421195140925745


In [121]:
gbm_result = gbm.predict_proba(X_challenge)

In [123]:
output = pd.read_csv("../data/test_format1.csv")
output.prob = gbm_result[:,1]
output.to_csv("../data/GBMresult.csv",index=False)

In [131]:
from sklearn.linear_model import LogisticRegression   


In [132]:
lr = LogisticRegression()

In [133]:
lr.fit(X_train, Y_train)
y_predprob = lr.predict_proba(X_test)
print(roc_auc_score(Y_test,y_predprob[:,1]))

0.43037640835054036


In [134]:
lr_result = lr.predict_proba(X_challenge)

In [135]:
output = pd.read_csv("../data/test_format1.csv")
output.prob = gbm_result[:,1]
output.to_csv("../data/LRresult.csv",index=False)